In [2]:
print("\n" + "="*90)
print(" 5. OPERASI LOGIKA (BOOLEAN LOGIC) ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     KONSEP MATEMATIKA DISKRIT                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Boolean logic untuk membuat ATURAN KEPUTUSAN berbasis kondisi diskrit.

OPERATOR LOGIKA:
  • ∧ (AND):  P ∧ Q - TRUE jika kedua kondisi TRUE
  • ∨ (OR):   P ∨ Q - TRUE jika salah satu TRUE
  • ¬ (NOT):  ¬P - negasi dari P
  • → (IMPLIES): P → Q - jika P maka Q

ATURAN KEPUTUSAN:

  1. KRITIS = (state = PUNCAK) ∨ (state = Peringatan ∧ increase > median)
     "Kondisi kritis: harga puncak ATAU peringatan dengan kenaikan tinggi"

  2. AMAN_BELI = (state = Normal) ∧ (week < -2)
     "Aman belanja: kondisi normal DAN masih jauh dari event"

  3. HINDARI = (state ∈ {PUNCAK, Peringatan}) ∧ (|week| ≤ 1)
     "Hindari belanja: kondisi krisis DAN dekat dengan event"
""")

# Hitung median per komoditas (lebih robust dari mean)
print("MENGHITUNG THRESHOLD LOGIKA:")
median_increase = df_all.groupby('commodity')['increase_pct'].transform('median')

# Aplikasikan operasi logika
df_all['high_increase'] = df_all['increase_pct'] > median_increase
df_all['pre_event'] = df_all['week'] < 0

# Kondisi komposit dengan operator logika
df_all['KRITIS'] = (df_all['state'] == 'PUNCAK') | \
                   ((df_all['state'] == 'Peringatan') & df_all['high_increase'])

df_all['AMAN_BELI'] = (df_all['state'] == 'Normal') & (df_all['week'] < -2)

df_all['HINDARI'] = (df_all['state'].isin(['PUNCAK', 'Peringatan'])) & \
                    (df_all['week'].abs() <= 1)

print(f"  Median increase per komoditas digunakan sebagai threshold adaptif")

print(f"\nHASIL EVALUASI PREDIKAT:")

logic_results = {
    'KRITIS': int(df_all['KRITIS'].sum()),
    'AMAN_BELI': int(df_all['AMAN_BELI'].sum()),
    'HINDARI': int(df_all['HINDARI'].sum())
}

total_obs = len(df_all)
for key, count in logic_results.items():
    pct = count / total_obs * 100
    print(f"  {key:12s}: {count:3d} kasus ({pct:5.1f}%)")

# Truth table untuk kondisi KRITIS
print(f"\nTRUTH TABLE (kondisi KRITIS):")
print(f"  Formula: (state=PUNCAK) ∨ (state=Peringatan ∧ increase>median)")
print()
print("  State       | Inc > Median |  PUNCAK  | Peringatan ∧ High | KRITIS")
print("  " + "-"*67)

truth_examples = [
    ('PUNCAK', True, True, False, True),
    ('PUNCAK', False, True, False, True),
    ('Peringatan', True, False, True, True),
    ('Peringatan', False, False, False, False),
    ('Normal', True, False, False, False),
]

for state, inc_high, is_peak, warn_high, result in truth_examples:
    print(f"  {state:12s}| {str(inc_high):10s}   | {str(is_peak):6s}   | {str(warn_high):15s}   | {str(result):6s}")

print("\n" + "="*80)
print("╔══════════════════════════════════════════════════════════════════════════════╗")
print("║                          APLIKASI DUNIA NYATA                                ║")
print("╚══════════════════════════════════════════════════════════════════════════════╝")

print("""
Sistem keputusan otomatis berbasis kombinasi kondisi diskrit.

Manfaat: Rekomendasi belanja otomatis dan alert system untuk kondisi kritis.
         Pedagang dapat setting alert jika komoditas masuk kondisi KRITIS,
         konsumen dapat planning belanja saat kondisi AMAN_BELI.
""")

if SHOW_EXAMPLE:
    print(f"\n📊 STUDI KASUS: SEMUA KOMODITAS")
    print(f"{'─'*80}")
    print(f"\n(Berdasarkan frekuensi kondisi KRITIS - Metrik Boolean Logic)")
    print(f"Analisis lengkap untuk semua {len(commodities_list)} komoditas tersedia di hasil evaluasi di atas.")

    # Hitung frekuensi KRITIS per komoditas
    kritis_by_commodity = df_all.groupby('commodity')['KRITIS'].agg(['sum', 'count'])
    kritis_by_commodity['pct'] = (kritis_by_commodity['sum'] / kritis_by_commodity['count'] * 100)
    kritis_by_commodity = kritis_by_commodity.sort_values('pct', ascending=False)

    # Filter yang ada di list
    valid_comms = [c for c in commodities_list if c in kritis_by_commodity.index]

    if len(valid_comms) >= 2:
        highest = kritis_by_commodity.loc[valid_comms].iloc[0]
        lowest = kritis_by_commodity.loc[valid_comms].iloc[-1]

        print(f"\n  {highest.name} (Paling Sering KRITIS):")
        print(f"    Frekuensi KRITIS: {int(highest['sum'])} dari {int(highest['count'])} observasi ({highest['pct']:.1f}%)")

        # Contoh data
        sample = df_all[(df_all['commodity'] == highest.name) & (df_all['KRITIS'])].head(1)
        if len(sample) > 0:
            row = sample.iloc[0]
            print(f"    Contoh: Minggu {row['week']:+.0f}, State={row['state']}, Increase={row['increase_pct']:.1f}%")

        print(f"\n  {lowest.name} (Paling Jarang KRITIS):")
        print(f"    Frekuensi KRITIS: {int(lowest['sum'])} dari {int(lowest['count'])} observasi ({lowest['pct']:.1f}%)")

        print(f"\n{'─'*80}")
        print(f"PENJELASAN:")
        print(f"  → {highest.name} sering masuk kondisi KRITIS (harga tinggi atau volatile)")
        print(f"  → {lowest.name} lebih stabil (jarang kondisi KRITIS)")
        print(f"  → Boolean logic memberikan alert otomatis untuk planning belanja")
        print(f"  → Sistem dapat trigger notifikasi saat kondisi KRITIS terpenuhi")

else:
    print(f"\n📊 STUDI KASUS: {COMMODITY_GROUPS[kategori_choice]['name'].upper()}")
    print(f"{'─'*80}")
    print(f"\nFrekuensi kondisi KRITIS untuk {len(commodities_list)} komoditas:")

    for commodity in commodities_list:
        comm_data = df_all[df_all['commodity'] == commodity]
        if len(comm_data) > 0:
            kritis_count = int(comm_data['KRITIS'].sum())
            total = len(comm_data)
            pct = kritis_count / total * 100
            print(f"\n  {commodity}:")
            print(f"    KRITIS: {kritis_count}/{total} observasi ({pct:.1f}%)")

    print(f"\n{'─'*80}")
    print(f"INSIGHT KATEGORI:")
    cat_kritis = df_all[df_all['commodity'].isin(commodities_list)]['KRITIS'].sum()
    cat_total = len(df_all[df_all['commodity'].isin(commodities_list)])
    cat_pct = cat_kritis / cat_total * 100
    print(f"  → Kategori {COMMODITY_GROUPS[kategori_choice]['name']}: {int(cat_kritis)}/{cat_total} KRITIS ({cat_pct:.1f}%)")
    print(f"  → Logic rules memberikan alert dini untuk kondisi kritis")

print("\n" + "="*80)

print(f"KESIMPULAN:")
print(f"  Boolean logic membuat 3 aturan keputusan otomatis dengan threshold adaptif.")
print(f"  Total kondisi KRITIS: {logic_results['KRITIS']} dari {total_obs} observasi.")

pause_section()

In [ ]:
print("\n" + "="*90)
print(" 6. KOMBINATORIKA (COMBINATORICS) ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     KONSEP MATEMATIKA DISKRIT                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Kombinatorika mempelajari cara menghitung dan menyusun objek diskrit.

FORMULA DASAR:

  Kombinasi (urutan tidak penting):
  C(n,r) = n! / [r!(n-r)!]

  Binomial Theorem:
  Σ[k=0 to n] C(n,k) = 2ⁿ

  Total kombinasi non-empty:
  Σ[k=1 to n] C(n,k) = 2ⁿ - 1

APLIKASI: Menghitung semua kemungkinan kombinasi faktor yang dapat
          menyebabkan kenaikan harga.

╔══════════════════════════════════════════════════════════════════════════════╗
║                          APLIKASI DUNIA NYATA                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Kenaikan harga sembako disebabkan kombinasi berbagai faktor.

Manfaat: Identifikasi semua skenario yang mungkin untuk contingency planning.
""")

# FAKTOR-FAKTOR PENYEBAB KENAIKAN HARGA
factors = ['Peningkatan Demand', 'Penurunan Supply', 'Kondisi Cuaca', 'Spekulasi Pasar']
n_factors = len(factors)

print("FAKTOR PENYEBAB KENAIKAN HARGA:")
for i, factor in enumerate(factors, 1):
    print(f"  {i}. {factor}")

print(f"\n  Total faktor: n = {n_factors}")

# HITUNG SEMUA KOMBINASI
print(f"\nANALISIS KOMBINASI C(n,k):")
print(f"  Berapa banyak cara faktor-faktor dapat berkombinasi?\n")

total_combinations = 0
combination_details = []

for k in range(1, n_factors + 1):
    num_comb = comb(n_factors, k)
    total_combinations += num_comb
    combination_details.append({'k': k, 'C(n,k)': num_comb})
    print(f"  k={k} ({k} faktor aktif bersamaan): C({n_factors},{k}) = {num_comb:2d} kombinasi")

print(f"\n  Total kombinasi non-empty: {total_combinations}")
print(f"  Verifikasi: 2^{n_factors} - 1 = {2**n_factors - 1} ✓")

# CONTOH KOMBINASI UNTUK TIAP k
print(f"\nCONTOH KOMBINASI UNTUK SETIAP k:")

# k=1
print(f"\n  k=1 (Satu faktor saja):")
for i, combo in enumerate(combinations(factors, 1), 1):
    print(f"    {i}. {combo[0]}")

# k=2
print(f"\n  k=2 (Dua faktor bersamaan):")
for i, combo in enumerate(combinations(factors, 2), 1):
    print(f"    {i}. {combo[0]} + {combo[1]}")

# k=3
print(f"\n  k=3 (Tiga faktor bersamaan):")
for i, combo in enumerate(combinations(factors, 3), 1):
    print(f"    {i}. {combo[0]} + {combo[1]} + {combo[2]}")

# k=4
print(f"\n  k=4 (Semua faktor aktif):")
combo = list(combinations(factors, 4))[0]
print(f"    1. {' + '.join(combo)}")

# INTERPRETASI
print(f"\nINTERPRETASI:")
print(f"  Ada {total_combinations} skenario berbeda yang perlu diantisipasi dalam")
print(f"  planning dan contingency management harga sembako.")
print()
print(f"  Skenario paling sederhana: 1 faktor aktif (4 kemungkinan)")
print(f"  Skenario paling kompleks: 4 faktor aktif (1 kemungkinan)")
print(f"  Skenario paling umum: 2 faktor aktif (6 kemungkinan)")

# KORELASI DENGAN DATA RIIL
print(f"\nKORELASI DENGAN DATA:")
kenaikan_tinggi = df_all[df_all['state'].isin(['Peringatan', 'PUNCAK'])]
kenaikan_rendah = df_all[df_all['state'] == 'Normal']

pct_tinggi = len(kenaikan_tinggi) / len(df_all) * 100
pct_rendah = len(kenaikan_rendah) / len(df_all) * 100

print(f"  Observasi kenaikan tinggi (Peringatan/PUNCAK): {len(kenaikan_tinggi)} ({pct_tinggi:.1f}%)")
print(f"  Observasi kenaikan rendah (Normal): {len(kenaikan_rendah)} ({pct_rendah:.1f}%)")
print()
print(f"  Hipotesis: Kenaikan tinggi kemungkinan melibatkan lebih banyak faktor")
print(f"             (k≥2), sedangkan kenaikan rendah hanya 1 faktor atau tidak ada.")

# Visualisasi
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# ========================================
# PLOT 1: Distribusi Kombinasi
# ========================================
k_values = list(range(1, n_factors + 1))
comb_values = [comb(n_factors, k) for k in k_values]

axes[0].bar(k_values, comb_values, color='steelblue', alpha=0.8,
            edgecolor='black', linewidth=2)
axes[0].set_xlabel('k (Jumlah Faktor Aktif)', fontsize=11, fontweight='bold')
axes[0].set_ylabel('C(4,k) - Jumlah Kombinasi', fontsize=11, fontweight='bold')
axes[0].set_title('Distribusi Kombinasi Faktor', fontsize=12, fontweight='bold')
axes[0].set_xticks(k_values)
axes[0].grid(True, alpha=0.3, axis='y')

for k, v in zip(k_values, comb_values):
    axes[0].text(k, v + 0.3, str(v), ha='center', fontweight='bold', fontsize=12)

# ========================================
# PLOT 2: Pascal's Triangle untuk C(n,k)
# ========================================
pascal_data = []
for n in range(5):
    row = [comb(n, k) for k in range(n + 1)]
    pascal_data.append(row)

# Plot sebagai heatmap
max_len = len(pascal_data[-1])
pascal_matrix = np.zeros((len(pascal_data), max_len))

for i, row in enumerate(pascal_data):
    start_idx = (max_len - len(row)) // 2
    for j, val in enumerate(row):
        pascal_matrix[i, start_idx + j] = val

# Mask nilai 0
masked_pascal = np.ma.masked_where(pascal_matrix == 0, pascal_matrix)

im = axes[1].imshow(masked_pascal, cmap='YlOrRd', aspect='auto')
axes[1].set_title("Pascal's Triangle (C(n,k))", fontsize=12, fontweight='bold')
axes[1].set_ylabel('n (jumlah faktor)', fontsize=11, fontweight='bold')
axes[1].set_xlabel('k (dipilih)', fontsize=11, fontweight='bold')
axes[1].set_yticks(range(5))
axes[1].set_xticks(range(max_len))

# Annotasi nilai
for i in range(len(pascal_data)):
    row = pascal_data[i]
    start_idx = (max_len - len(row)) // 2
    for j, val in enumerate(row):
        if val > 0:
            axes[1].text(start_idx + j, i, int(val), ha='center', va='center',
                        fontweight='bold', fontsize=10, color='black')

# Highlight n=4 (kasus kita)
axes[1].axhline(y=4, color='red', linestyle='--', linewidth=2, alpha=0.7)
axes[1].text(max_len-0.5, 4, 'n=4 (kasus kita)', fontsize=9,
            ha='right', va='center', color='red', fontweight='bold')

# ========================================
# PLOT 3: Pie Chart Distribusi State
# ========================================
state_counts = df_all['state'].value_counts()
colors_pie = {'Normal': '#2ecc71', 'Siaga': '#f39c12',
              'Peringatan': '#e67e22', 'PUNCAK': '#e74c3c'}
colors = [colors_pie.get(s, 'gray') for s in state_counts.index]

wedges, texts, autotexts = axes[2].pie(state_counts.values, labels=state_counts.index,
                                        autopct='%1.1f%%', colors=colors, startangle=90,
                                        explode=[0.05]*len(state_counts),
                                        textprops={'fontsize': 10, 'fontweight': 'bold'})

# Tambahkan legenda dengan count
legend_labels = [f"{state}: {count}" for state, count in state_counts.items()]
axes[2].legend(legend_labels, loc='upper left', bbox_to_anchor=(1, 1), fontsize=9)

axes[2].set_title('Distribusi State dalam Data\n(Hipotesis: State tinggi → Faktor lebih banyak)',
                  fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nKESIMPULAN:")
print(f"  Kombinatorika mengidentifikasi {total_combinations} skenario kombinasi faktor.")
print(f"  Setiap skenario memerlukan strategi mitigasi yang berbeda.")

pause_section()

In [ ]:

print("\n" + "="*90)
print(" 7. TEORI BILANGAN (NUMBER THEORY) ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     KONSEP MATEMATIKA DISKRIT                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Teori bilangan mempelajari sifat bilangan bulat dan operasinya.

KONSEP YANG DIGUNAKAN:

1. MODULAR ARITHMETIC:
   a mod n = sisa pembagian a dengan n
   Aplikasi: Analisis pola berdasarkan relasi ekuivalensi modulo

2. GCD (Greatest Common Divisor):
   GCD(a,b) = faktor persekutuan terbesar
   Algoritma Euclidean: GCD(a, b) = GCD(b, a mod b)
   Aplikasi: Siklus umum timing peak

3. DIVISIBILITY:
   a | b (a membagi b) jika b mod a = 0
   Aplikasi: Unit perubahan harga (kelipatan Rp 100, Rp 500, dll)
""")
print("PERSIAPAN DATA UNTUK TEORI BILANGAN:")
print("  Membulatkan harga ke Rupiah terdekat (data asli desimal)")

# PERSIAPAN: Bulatkan harga ke integer (Rupiah)
print("PERSIAPAN DATA UNTUK TEORI BILANGAN:")
print("  Membulatkan harga ke Rupiah terdekat")

df_all['price_int'] = df_all['price'].round(0).astype(int)
print(f"  ✓ Total observasi: {len(df_all)}")

# ANALISIS 1: Relasi Ekuivalensi Modulo (RINGKAS - PERBAIKAN)
print(f"\nRELASI EKUIVALENSI MODULO:")

df_all['week_mod_3'] = df_all['week'] % 3

# Buat dataframe lengkap untuk visualisasi
price_by_mod = df_all.groupby('week_mod_3').agg({
    'increase_pct': 'mean'
}).round(2)

max_diff = price_by_mod['increase_pct'].max() - price_by_mod['increase_pct'].min()
print(f"  Rentang perbedaan: {max_diff:.2f}%")
if max_diff < 3:
    print(f"  → Pola modulo 3 TIDAK signifikan")
else:
    print(f"  → Ada pola modulo 3 signifikan")


# ANALISIS 2: GCD & LCM
print(f"\nALGORITMA EUCLIDEAN - GCD & LCM:")

peak_weeks_by_commodity = {}
for commodity in commodities_list:
    mask = df_all['commodity'] == commodity
    peak_idx = df_all[mask]['increase_pct'].idxmax()
    if pd.notna(peak_idx):
        peak_week = abs(int(df_all.loc[peak_idx, 'week']))
        if peak_week > 0:
            peak_weeks_by_commodity[commodity] = peak_week

if len(peak_weeks_by_commodity) >= 2:
    peak_weeks_list = list(peak_weeks_by_commodity.values())
    overall_gcd = reduce(gcd, peak_weeks_list)

    def lcm(a, b):
        return abs(a * b) // gcd(a, b) if a and b else 0

    overall_lcm = reduce(lcm, peak_weeks_list)

    print(f"  GCD = {overall_gcd}, LCM = {overall_lcm}")
    if overall_gcd > 1:
        print(f"  → Siklus {overall_gcd}-minggu terdeteksi")
    else:
        print(f"  → Tidak ada siklus umum")
else:
    overall_gcd = 1
    overall_lcm = 1
    print("  → Data tidak cukup untuk GCD/LCM")

# ANALISIS 3: Divisibilitas
print(f"\nDIVISIBILITY:")

df_all_sorted = df_all.sort_values(['commodity', 'event', 'week']).copy()
df_all_sorted['price_change_int'] = df_all_sorted.groupby(['commodity', 'event'])['price_int'].diff()

price_changes_abs = df_all_sorted['price_change_int'].dropna().abs()
price_changes_abs = price_changes_abs[price_changes_abs > 0].astype(int)

divisors = [10, 50, 100, 500, 1000]
divisibility_stats = {}

for divisor in divisors:
    count = (price_changes_abs % divisor == 0).sum()
    pct = (count / len(price_changes_abs) * 100)
    divisibility_stats[divisor] = {'count': int(count), 'percentage': pct}

# Tampilkan hanya yang > 10%
print(f"  Kelipatan signifikan:")
for divisor in divisors:
    if divisibility_stats[divisor]['percentage'] > 10:
        print(f"    Rp {divisor}: {divisibility_stats[divisor]['percentage']:.1f}%")

# Interpretasi singkat
if divisibility_stats[100]['percentage'] > 50:
    print(f"  → Pedagang gunakan unit Rp 100")
elif divisibility_stats[10]['percentage'] > 50:
    print(f"  → Pedagang gunakan unit Rp 10")
else:
    print(f"  → Perubahan harga presisi (non-kelipatan)")


print("\n" + "="*80)
print("╔══════════════════════════════════════════════════════════════════════════════╗")
print("║                          APLIKASI DUNIA NYATA                                ║")
print("╚══════════════════════════════════════════════════════════════════════════════╝")

print("""
Identifikasi pola tersembunyi menggunakan teori bilangan bulat dan modular arithmetic.

Manfaat: Deteksi siklus berulang untuk prediksi timing peak (GCD memberikan interval umum).
         Pedagang dapat gunakan strategi pricing berbasis kelipatan tertentu (Rp 100, Rp 500)
         untuk memudahkan perhitungan dan konsisten dengan pola pasar.
         Analisis modulo mengidentifikasi apakah ada pola periodik dalam kenaikan harga.
""")

if SHOW_EXAMPLE:
    print(f"\n📊 STUDI KASUS: SEMUA KOMODITAS")
    print(f"{'─'*80}")
    print(f"\n(Berdasarkan divisibility pattern - Metrik Teori Bilangan)")
    print(f"Analisis lengkap untuk semua {len(commodities_list)} komoditas tersedia di analisis di atas.")

    # Hitung divisibility Rp 100 per komoditas
    commodity_div_stats = []

    for commodity in commodities_list:
        comm_data = df_all_sorted[df_all_sorted['commodity'] == commodity].copy()
        comm_changes = comm_data['price_change_int'].dropna().abs()
        comm_changes = comm_changes[comm_changes > 0].astype(int)

        if len(comm_changes) > 0:
            div_100 = (comm_changes % 100 == 0).sum()
            pct_100 = (div_100 / len(comm_changes) * 100)

            commodity_div_stats.append({
                'commodity': commodity,
                'div_100_pct': pct_100,
                'total_changes': len(comm_changes)
            })

    if len(commodity_div_stats) >= 2:
        div_df = pd.DataFrame(commodity_div_stats).sort_values('div_100_pct', ascending=False)

        highest = div_df.iloc[0]
        lowest = div_df.iloc[-1]

        print(f"\n  {highest['commodity']} (Paling Konsisten - Kelipatan Rp 100):")
        print(f"    Perubahan kelipatan Rp 100: {highest['div_100_pct']:.1f}%")
        print(f"    Total perubahan: {highest['total_changes']}")

        print(f"\n  {lowest['commodity']} (Paling Fleksibel - Perubahan Halus):")
        print(f"    Perubahan kelipatan Rp 100: {lowest['div_100_pct']:.1f}%")
        print(f"    Total perubahan: {lowest['total_changes']}")

        print(f"\n{'─'*80}")
        print(f"PENJELASAN:")
        print(f"  → {highest['commodity']} pakai pricing strategy kelipatan besar (mudah dihitung)")
        print(f"  → {lowest['commodity']} pakai pricing lebih presisi (sesuai supply-demand)")
        print(f"  → Divisibility tinggi = pricing lebih terstruktur dan konsisten")
        print(f"  → GCD = {overall_gcd} menunjukkan siklus umum timing peak")

else:
    print(f"\n📊 STUDI KASUS: {COMMODITY_GROUPS[kategori_choice]['name'].upper()}")
    print(f"{'─'*80}")
    print(f"\nDivisibility pattern untuk {len(commodities_list)} komoditas:")

    for commodity in commodities_list:
        comm_data = df_all_sorted[df_all_sorted['commodity'] == commodity].copy()
        comm_changes = comm_data['price_change_int'].dropna().abs()
        comm_changes = comm_changes[comm_changes > 0].astype(int)

        if len(comm_changes) > 0:
            div_100 = (comm_changes % 100 == 0).sum()
            pct_100 = (div_100 / len(comm_changes) * 100)
            print(f"\n  {commodity}:")
            print(f"    Kelipatan Rp 100: {pct_100:.1f}%")

    print(f"\n{'─'*80}")
    print(f"INSIGHT KATEGORI:")
    cat_changes = df_all_sorted[df_all_sorted['commodity'].isin(commodities_list)]['price_change_int'].dropna().abs()
    cat_changes = cat_changes[cat_changes > 0].astype(int)

    if len(cat_changes) > 0:
        cat_div_100 = (cat_changes % 100 == 0).sum()
        cat_pct_100 = (cat_div_100 / len(cat_changes) * 100)
        print(f"  → Kategori {COMMODITY_GROUPS[kategori_choice]['name']}: {cat_pct_100:.1f}% kelipatan Rp 100")

        if cat_pct_100 > 50:
            print(f"  → Pricing strategy kategori: TERSTRUKTUR (kelipatan besar)")
        else:
            print(f"  → Pricing strategy kategori: PRESISI (perubahan halus)")

print("\n" + "="*80)

# Visualisasi
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Kenaikan per kelas ekuivalensi mod 3
axes[0].bar(price_by_mod.index, price_by_mod['increase_pct'].values,
            color='steelblue', alpha=0.8, edgecolor='black', linewidth=2)
axes[0].axhline(y=0, color='red', linestyle='-', linewidth=1)
axes[0].set_xlabel('Week mod 3', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Rata-rata Kenaikan (%)', fontsize=11, fontweight='bold')
axes[0].set_title('Relasi Ekuivalensi: Week ≡ k (mod 3)', fontsize=12, fontweight='bold')
axes[0].set_xticks([0, 1, 2])
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: GCD timing peak
if len(peak_weeks_by_commodity) >= 5:
    sample_comms = list(peak_weeks_by_commodity.keys())[:5]
    sample_weeks = [peak_weeks_by_commodity[c] for c in sample_comms]

    axes[1].barh(sample_comms, sample_weeks, color='green', alpha=0.8,
                edgecolor='black', linewidth=1.5)
    if overall_gcd > 1:
        axes[1].axvline(x=overall_gcd, color='red', linestyle='--', linewidth=2.5,
                       label=f'GCD = {overall_gcd}', alpha=0.8)
    axes[1].set_xlabel('Jarak Peak dari Event (minggu)', fontsize=11, fontweight='bold')
    axes[1].set_title('Timing Peak & GCD', fontsize=12, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3, axis='x')
else:
    axes[1].text(0.5, 0.5, 'Data tidak cukup', ha='center', va='center', fontsize=12)
    axes[1].set_title('Timing Peak & GCD', fontsize=12, fontweight='bold')
    axes[1].axis('off')

# Plot 3: Divisibilitas
available_divisors = [d for d in divisors if d in divisibility_stats]
div_labels = [f'Rp {d}' for d in available_divisors]
div_percentages = [divisibility_stats[d]['percentage'] for d in available_divisors]

axes[2].bar(div_labels, div_percentages, color='purple', alpha=0.8,
            edgecolor='black', linewidth=2)
axes[2].set_xlabel('Pembagi', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Persentase (%)', fontsize=11, fontweight='bold')
axes[2].set_title('Divisibilitas Perubahan Harga', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\nKESIMPULAN:")
print(f"  Teori bilangan mengidentifikasi:")
print(f"    • GCD timing = {overall_gcd} (siklus umum)")
print(f"    • Divisibilitas terbanyak: Rp {max(divisors[1:], key=lambda k: divisibility_stats[k]['percentage'])}")
print(f"    • Pola modulo 3 {'signifikan' if max_diff >= 3 else 'tidak signifikan'}")

pause_section()

In [ ]:
print("\n" + "="*90)
print(" 8. TEORI HIMPUNAN (SET THEORY) ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     KONSEP MATEMATIKA DISKRIT                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Teori himpunan mempelajari koleksi objek diskrit dan operasinya.

DEFINISI DASAR:
  • Himpunan A = {x | x memenuhi properti P}
  • Kardinalitas: |A| = jumlah elemen di A

OPERASI HIMPUNAN:
  • Intersection (A ∩ B): {x | x ∈ A ∧ x ∈ B}
  • Union (A ∪ B): {x | x ∈ A ∨ x ∈ B}
  • Complement (Aᶜ): {x | x ∈ U ∧ x ∉ A}
  • Difference (A − B): {x | x ∈ A ∧ x ∉ B}

HUKUM SET THEORY:
  • |A ∪ B| = |A| + |B| - |A ∩ B|
  • (A ∩ B)ᶜ = Aᶜ ∪ Bᶜ (De Morgan's Law)
""")

# DEFINISI THRESHOLD DETERMINISTIK
spike_threshold = df_all.groupby('commodity')['increase_pct'].quantile(0.75).mean()

print(f"DEFINISI HIMPUNAN:")
print(f"  U = semua komoditas (|U| = {len(commodities_list)})")
print(f"  Threshold spike: ≥ {spike_threshold:.1f}% (rata-rata Q3)")

# BANGUN HIMPUNAN PER EVENT
print(f"\nHIMPUNAN SPIKE PER EVENT:")

event_sets = {}
for event in events:
    event_spikes = set()
    for commodity in commodities_list:
        mask = (df_all['commodity'] == commodity) & (df_all['event'] == event)
        max_increase = df_all[mask]['increase_pct'].max()
        if max_increase >= spike_threshold:
            event_spikes.add(commodity)

    event_sets[event] = event_spikes
    print(f"  S_{event}: |S| = {len(event_spikes)}")
    if len(event_spikes) <= 5:
        print(f"    {event_spikes if event_spikes else 'Ø'}")
    else:
        sample = list(event_spikes)[:3]
        print(f"    {{{', '.join(sample)}, ...}}")

# OPERASI HIMPUNAN
print(f"\nOPERASI HIMPUNAN:")

# Intersection
if len(events) == 2:
    intersection = event_sets[events[0]] & event_sets[events[1]]
    print(f"  A ∩ B (spike KEDUA event): {intersection if intersection else 'Ø'}")
    print(f"    |A ∩ B| = {len(intersection)}")
else:
    intersection = set.intersection(*event_sets.values()) if event_sets else set()
    print(f"  ∩ (spike SEMUA event): {intersection if intersection else 'Ø'}")
    print(f"    |∩| = {len(intersection)}")

# Union
union = set.union(*event_sets.values()) if event_sets else set()
print(f"\n  A ∪ B (spike MINIMAL 1 event):")
if len(union) <= 7:
    print(f"    {union}")
else:
    sample = list(union)[:5]
    print(f"    {{{', '.join(sample)}, ...}}")
print(f"    |A ∪ B| = {len(union)}")

# Complement
complement = set(commodities_list) - union
print(f"\n  (A ∪ B)ᶜ (TIDAK PERNAH spike): {complement if complement else 'Ø'}")
print(f"    |(A ∪ B)ᶜ| = {len(complement)}")

# Difference (jika 2 event)
if len(events) == 2:
    diff_AB = event_sets[events[0]] - event_sets[events[1]]
    diff_BA = event_sets[events[1]] - event_sets[events[0]]

    print(f"\n  A − B: {diff_AB if diff_AB else 'Ø'} (|A − B| = {len(diff_AB)})")
    print(f"  B − A: {diff_BA if diff_BA else 'Ø'} (|B − A| = {len(diff_BA)})")

# VERIFIKASI HUKUM
print(f"\nVERIFIKASI HUKUM:")

if len(events) == 2:
    card_A = len(event_sets[events[0]])
    card_B = len(event_sets[events[1]])
    card_union = len(union)
    card_intersection = len(intersection)
    expected = card_A + card_B - card_intersection

    print(f"  |A ∪ B| = |A| + |B| - |A ∩ B|")
    print(f"  {card_union} = {card_A} + {card_B} - {card_intersection} = {expected} {'✓' if card_union == expected else '✗'}")

card_U = len(commodities_list)
print(f"  |U| = |A ∪ B| + |(A ∪ B)ᶜ|")
print(f"  {card_U} = {len(union)} + {len(complement)} = {len(union) + len(complement)} {'✓' if card_U == len(union) + len(complement) else '✗'}")

print("\n" + "="*80)
print("╔══════════════════════════════════════════════════════════════════════════════╗")
print("║                          APLIKASI DUNIA NYATA                                ║")
print("╚══════════════════════════════════════════════════════════════════════════════╝")

print(f"""
Pengelompokan komoditas berdasarkan volatilitas harga menggunakan operasi himpunan.

Manfaat: Prioritas monitoring komoditas yang konsisten volatile (A ∩ B - spike di semua event).
         Pedagang fokus stok komoditas di intersection (risiko tinggi), abaikan complement (stabil).
         Set operations mengidentifikasi pola volatilitas: {len(intersection)} komoditas spike konsisten,
         {len(complement)} komoditas selalu stabil, dan {len(union)} total volatile.
""")

if SHOW_EXAMPLE:
    print(f"\n📊 STUDI KASUS: SEMUA KOMODITAS")
    print(f"{'─'*80}")
    print(f"\n(Berdasarkan set membership - Metrik Teori Himpunan)")
    print(f"Analisis lengkap untuk semua {len(commodities_list)} komoditas tersedia di operasi himpunan di atas.")

    # Hitung berapa kali komoditas spike (muncul di berapa event)
    commodity_spike_count = {}
    for commodity in commodities_list:
        count = sum(1 for event_set in event_sets.values() if commodity in event_set)
        commodity_spike_count[commodity] = count

    # Sort
    sorted_comms = sorted(commodity_spike_count.items(), key=lambda x: x[1], reverse=True)

    if len(sorted_comms) >= 2:
        most_volatile = sorted_comms[0]
        most_stable = sorted_comms[-1]

        print(f"\n  {most_volatile[0]} (Paling Volatile):")
        print(f"    Spike di {most_volatile[1]}/{len(events)} event")
        if most_volatile[1] == len(events):
            print(f"    Status: Masuk A ∩ B (intersection) - PRIORITAS TINGGI")
        else:
            print(f"    Status: Masuk A ∪ B (union) tapi tidak intersection")

        print(f"\n  {most_stable[0]} (Paling Stabil):")
        print(f"    Spike di {most_stable[1]}/{len(events)} event")
        if most_stable[1] == 0:
            print(f"    Status: Masuk (A ∪ B)ᶜ (complement) - AMAN DIABAIKAN")
        else:
            print(f"    Status: Masuk A ∪ B tapi jarang spike")

        print(f"\n{'─'*80}")
        print(f"PENJELASAN:")
        print(f"  → Komoditas di intersection (spike semua event) = risiko tertinggi")
        print(f"  → Komoditas di complement (tidak pernah spike) = paling stabil")
        print(f"  → Operasi himpunan membagi komoditas ke kategori risiko yang jelas")

else:
    print(f"\n📊 STUDI KASUS: {COMMODITY_GROUPS[kategori_choice]['name'].upper()}")
    print(f"{'─'*80}")
    print(f"\nSet membership untuk {len(commodities_list)} komoditas:")

    for commodity in commodities_list:
        count = sum(1 for event_set in event_sets.values() if commodity in event_set)

        if count == len(events):
            status = "A ∩ B (intersection)"
        elif count == 0:
            status = "(A ∪ B)ᶜ (complement)"
        else:
            status = "A ∪ B (union only)"

        print(f"\n  {commodity}:")
        print(f"    Spike: {count}/{len(events)} event")
        print(f"    Status: {status}")

    print(f"\n{'─'*80}")
    print(f"INSIGHT KATEGORI:")
    cat_intersection = sum(1 for c in commodities_list if c in intersection)
    cat_complement = sum(1 for c in commodities_list if c in complement)
    print(f"  → Kategori {COMMODITY_GROUPS[kategori_choice]['name']}:")
    print(f"    {cat_intersection} di intersection (konsisten volatile)")
    print(f"    {cat_complement} di complement (konsisten stabil)")

print("\n" + "="*80)

# Visualisasi Venn Diagram (untuk 2 event)
if len(events) == 2:
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    # Parameter lingkaran
    circle_A = plt.Circle((0.35, 0.5), 0.3, color='blue', alpha=0.3, label=events[0])
    circle_B = plt.Circle((0.65, 0.5), 0.3, color='red', alpha=0.3, label=events[1])

    ax.add_patch(circle_A)
    ax.add_patch(circle_B)

    # Label region
    # A only
    if diff_AB:
        ax.text(0.25, 0.5, f"A - B\n|{len(diff_AB)}|", ha='center', va='center',
                fontsize=11, fontweight='bold')
        sample_text = '\n'.join(list(diff_AB)[:3])
        ax.text(0.25, 0.35, sample_text, ha='center', va='top', fontsize=8)

    # Intersection
    if intersection:
        ax.text(0.5, 0.5, f"A ∩ B\n|{len(intersection)}|", ha='center', va='center',
                fontsize=11, fontweight='bold', color='darkgreen')
        sample_text = '\n'.join(list(intersection)[:3])
        ax.text(0.5, 0.35, sample_text, ha='center', va='top', fontsize=8)
    else:
        ax.text(0.5, 0.5, "A ∩ B\nØ", ha='center', va='center',
                fontsize=11, fontweight='bold', color='gray')

    # B only
    if diff_BA:
        ax.text(0.75, 0.5, f"B - A\n|{len(diff_BA)}|", ha='center', va='center',
                fontsize=11, fontweight='bold')
        sample_text = '\n'.join(list(diff_BA)[:3])
        ax.text(0.75, 0.35, sample_text, ha='center', va='top', fontsize=8)

    # Complement
    if complement:
        ax.text(0.15, 0.15, f"(A ∪ B)ᶜ\n|{len(complement)}|", ha='center', va='center',
                fontsize=10, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        sample_text = '\n'.join(list(complement)[:2])
        ax.text(0.15, 0.05, sample_text, ha='center', va='top', fontsize=7)

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Venn Diagram: Komoditas Spike per Event', fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='upper right', fontsize=11)

    plt.tight_layout()
    plt.show()

print(f"\nKESIMPULAN:")
print(f"  Teori himpunan mengidentifikasi:")
print(f"    • {len(intersection)} komoditas spike di semua event (prioritas monitoring)")
print(f"    • {len(complement)} komoditas tidak pernah spike (stabil)")
print(f"    • {len(union)} total komoditas yang pernah spike")

pause_section()